# Causal Identification in DAGs using Backdoor and Swigs, Equivalence Classes, Falsifiability Tests


In [ ]:
%pip install rpy2
%load_ext rpy2.ipython
import numpy as np
import statsmodels.formula.api as smf
import rpy2
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

In [ ]:
%%R
install.packages("dagitty")
install.packages("ggdag")
library(dagitty)
library(ggdag)

# Graph Generation and Plotting 

The following DAG is due to Judea Pearl

In [ ]:
%%R
G = dagitty('dag{
Z1 [pos="-2,-1.5"]
X1 [pos="-2,0"]
Z2 [pos="1.5,-1.5"]
X3 [pos="1.5, 0"]
Y [outcome,pos="1.5,1.5"]
D [exposure,pos="-2,1.5"]
M [mediator, pos="0,1.5"]
X2 [pos="0,0"]
Z1 -> X1
X1 -> D
Z1 -> X2
Z2 -> X3
X3 -> Y
Z2 -> X2
D -> Y
X2 -> Y
X2 -> D
M->Y
D->M
}')


ggdag(G)+  theme_dag()

# Report Relatives of X2

In [ ]:
%%R
print(parents(G, "X2"))
print(children(G, "X2"))
print(ancestors(G, "X2"))
print(descendants(G, "X2"))

# Find Paths Between D and Y



In [ ]:
%%R
paths(G, "D", "Y")

# List All Testable Implications of the Model

In [ ]:
%%R
print(impliedConditionalIndependencies(G))

# Identification by Backdoor: List minimal adjustment sets to identify causal effecs $D \to Y$

In [ ]:
%%R
print(adjustmentSets( G, "D", "Y" ))

# Identification via SWIG and D-separation

In [ ]:
%%R
SWIG = dagitty('dag{
Z1 [pos="-2,-1.5"]
X1 [pos="-2,0"]
Z2 [pos="1.5,-1.5"]
X3 [pos="1.5, 0"]
Yd [outcome,pos="1.5,1.5"]
D [exposure,pos="-2,1.5"]
d [pos="-1, 1.5"]
Md [mediator, pos="0,1.5"]
X2 [pos="0,0"]
Z1 -> X1
X1 -> D
Z1 -> X2
Z2 -> X3
X3 -> Yd
Z2 -> X2
X2 -> Yd
X2 -> D
X3-> Yd
Md-> Yd
d-> Md
}')

ggdag(SWIG)+  theme_dag()


# Deduce Conditional Exogeneity or Ignorability by D-separation


In [ ]:
%%R
print(impliedConditionalIndependencies(SWIG)[5:8])

This coincides with the backdoor criterion for this graph.

# Print All Average Effects Identifiable by Conditioning

In [ ]:
%%R 
for( n in names(G) ){
    for( m in children(G,n) ){
        a <- adjustmentSets( G, n, m )
        if( length(a) > 0 ){
            cat("The effect ",n,"->",m,
                " is identifiable by controlling for:\n",sep="")
            print( a, prefix=" * " )
        }
    }
}

# Equivalence Classes

In [ ]:
%%R
P=equivalenceClass(G)
plot(P)

Next Consider the elemntary Triangular Model:
$$
D \to Y, \quad X \to (D,Y).
$$
This model has not testable implications and is Markov-equivalent to any other DAG difined on names $(X, D, Y)$.

In [ ]:
%%R
G3<- dagitty('dag{
D -> Y
X -> D
X -> Y
}
')
ggdag(G3)+  theme_dag()
print(impliedConditionalIndependencies(G3))

In [ ]:
%%R
P=equivalenceClass(G3)
plot(P)
equivalentDAGs(G3,10)

# Example of Testing DAG Validity

Next we simulate the data from a Linear SEM associated to DAG G, and perform a test of conditional independence restrictions, exploting linearity. 


There are many other options for nonlinear models and discrete categorical variabales. Type help(localTests). 

In [ ]:
%%R
x <- simulateSEM(G)
localTests(G, data = x, type = c("cis"))

Next we replaced $D$ by $\bar D$ generated differently:
$$
\bar D= (D + Y)/2
$$
So basically $\bar D$ is an average of $D$ and $Y$ generated by $D$.  We then test if the resulting collection of random variables satisifes conditional indepdendence restrictions, exploiting linearity.  We end up rejectiong these restrictions and thefore the validity of this model for the data generated in this way.  This makes sense, because the new data no longer obeys the previous DAG structure.



In [ ]:
%%R
x.R <- x
x.R$D <- (x$D+ x$Y)/2
localTests(G, data = x.R, type = c("cis"))